In [1]:
import os
import nest_asyncio
from flask import Flask, request, send_file, render_template_string
import googlemaps
import pandas as pd
from threading import Thread
import logging
import time

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Initialize Flask app and Google Maps client
app = Flask(__name__)
gmaps = googlemaps.Client(key='Your-Api-Key')

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the HTML template
HTML_TEMPLATE = '''
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Google Maps Places Search</title>
    <style>
      body {
        font-family: Arial, sans-serif;
        background-color: #f4f4f9;
        margin: 0;
        padding: 0;
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
      }
      .container {
        background-color: white;
        padding: 2em;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        border-radius: 10px;
        text-align: center;
      }
      h1 {
        color: #333;
        margin-bottom: 1em;
      }
      label {
        display: block;
        margin-bottom: 0.5em;
        color: #555;
      }
      input[type="text"] {
        width: 100%;
        padding: 0.5em;
        margin-bottom: 1em;
        border: 1px solid #ddd;
        border-radius: 5px;
        box-sizing: border-box;
      }
      button {
        background-color: #007bff;
        color: white;
        padding: 0.75em 1.5em;
        border: none;
        border-radius: 5px;
        cursor: pointer;
        font-size: 1em;
      }
      button:hover {
        background-color: #0056b3;
      }
    </style>
  </head>
  <body>
    <div class="container">
      <h1>Google Maps Places Search</h1>
      <form action="/search" method="post">
        <label for="query">Enter Search Query:</label>
        <input type="text" id="query" name="query" required>
        <button type="submit">Search and Download Excel</button>
      </form>
    </div>
  </body>
</html>
'''

@app.route('/')
def index():
    logger.info("Rendering index page")
    return render_template_string(HTML_TEMPLATE)

@app.route('/search', methods=['POST'])
def search():
    query = request.form['query']
    logger.info(f"Received query: {query}")
    results = []
    try:
        next_page_token = None
        while len(results) < 100:
            places_result = gmaps.places(query=query, page_token=next_page_token)
            logger.info(f"Fetched {len(places_result['results'])} places")
            
            for place in places_result['results']:
                name = place.get('name', 'N/A')
                address = place.get('formatted_address', 'N/A')
                place_id = place.get('place_id', None)
                
                phone_number = 'N/A'
                website = 'N/A'
                email = 'N/A'
                
                if place_id:
                    place_details = gmaps.place(place_id=place_id)
                    if place_details.get('result'):
                        phone_number = place_details['result'].get('formatted_phone_number', 'N/A')
                        website = place_details['result'].get('website', 'N/A')
                        email = place_details['result'].get('email', 'N/A')
                
                results.append({
                    'Name': name,
                    'Address': address,
                    'Phone Number': phone_number,
                    'Website': website,
                    'Email': email
                })
                
                if len(results) >= 100:
                    break
            
            next_page_token = places_result.get('next_page_token', None)
            if not next_page_token:
                break
            logger.info("Waiting for next page token")
            time.sleep(2)  # Add delay to ensure the next_page_token is ready
        
        logger.info(f"Collected {len(results)} places")

        # Create a DataFrame and save to Excel
        df = pd.DataFrame(results)
        excel_file = "places_results.xlsx"
        df.to_excel(excel_file, index=False)
        
        return send_file(excel_file, as_attachment=True, download_name=excel_file)

    except googlemaps.exceptions.ApiError as e:
        logger.error(f"API error: {e}")
        return f"API error: {e}", 500
    except googlemaps.exceptions.TransportError as e:
        logger.error(f"Transport error: {e}")
        return f"Transport error: {e}", 500
    except googlemaps.exceptions.Timeout as e:
        logger.error(f"Timeout error: {e}")
        return f"Timeout error: {e}", 500
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        return f"Unexpected error: {e}", 500

# Function to run Flask app
def run_app():
    app.run(port=8000, use_reloader=False)

# Run Flask app in a separate thread
thread = Thread(target=run_app)
thread.start()

# Now, the Flask app should be running and accessible at http://127.0.0.1:8000/
logger.info("Flask app is running at http://127.0.0.1:8000/")


INFO:__main__:Flask app is running at http://127.0.0.1:8000/


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Rendering index page
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 12:37:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 12:37:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:__main__:Received query: rentals in banglore
INFO:__main__:Fetched 0 places
INFO:__main__:Collected 0 places
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 12:37:38] "POST /search HTTP/1.1" 200 -
INFO:__main__:Received query: rentals in banglore
INFO:__main__:Fetched 0 places
INFO:__main__:Collected 0 places
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 12:39:04] "POST /search HTTP/1.1" 200 -
INFO:__main__:Received query: bike rentals in banglore
INFO:__main__:Fetched 20 places
INFO:__main__:Waiting for next page token
INFO:__main__:Fetched 20 places
INFO:__main__:Waiting for next page token
INFO:__ma